In [319]:
from decimal import Decimal
from string import ascii_lowercase
import datetime
from collections import namedtuple

class TimeZone:
    def __init__(self, offset, name):
        self.name = name
        self.offset = offset
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise ValueError('Name must be a string.')
        if not value.strip():
            raise ValueError('Name must not be blank.')
        self._name = value.strip()
        
    @property
    def offset(self):
        return self._offset
    
    @offset.setter
    def offset(self, value):
        if not isinstance(value, datetime.timedelta):
            raise TypeError('Offset must be a datetime.timedelta object.')
        if value < datetime.timedelta(hours=-12, minutes=0) or value > datetime.timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00.')
        self._offset = value
        
    def __repr__(self):
        return f'TimeZone(offset={self.offset}, name={self.name})'
        
class BankAccount:
    _monthly_interest_rate = None
    _tran_id = 0
    
    _transaction_codes = {
        'deposit': 'D',
        'withdrawal': 'W',
        'interest': 'I',
        'rejected': 'X'
    }
    
    def __init__(self, accnt_no, f_name, l_name, tz_offset=None, balance=None):
        self.accnt_no = accnt_no
        self.f_name = f_name
        self.l_name = l_name
        self.tz_offset = tz_offset
        self._balance = Decimal(0.00)
        if balance:
            self.deposit(balance)

    @staticmethod
    def _validate_name(value, field_title):
        # If every character in the provided value (assuming it's a string) is not a
        # standard letter, a space, or an apostrophe, raise an exception.
        if len(str(value).strip()) == 0 or value is None:
            raise ValueError(f'{field_title} must not be blank.')
        if not all(map(lambda x: x.lower() in ascii_lowercase + " '", str(value))):
            raise ValueError(f'{field_title} must contain only letters, spaces, or apostrophes.')
        return str(value).strip()
            
    @staticmethod
    def parse_conf_no(conf_no, timezone):
        ConfirmationNumber = namedtuple('ConfirmationNumber',
                                        ['account_number',
                                         'transaction_code',
                                         'transaction_id',
                                         'time',
                                         'time_utc']
                                       )
        
        conf_components = conf_no.split('-')
        tran_code = conf_components[0]
        accnt_no = conf_components[1]
        time_utc = datetime.datetime.strptime(conf_components[2], '%Y%m%d%H%M%S')
        tran_id = conf_components[3]
        time = f'{time_utc + timezone.offset} ({timezone.name})'
        
        parsed_conf_no = ConfirmationNumber(accnt_no,
                                            tran_code,
                                            tran_id,
                                            time,
                                            time_utc.isoformat())
        return parsed_conf_no
            
    @classmethod
    def get_interest_rate(cls):
        if cls._monthly_interest_rate is None:
            raise ValueError('Interest rate has not been set.')
        return cls._monthly_interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        try:
            value = Decimal(value)
        except:
            raise ValueError('Interest rate must be a decimal value.')
        if value < 0:
            raise ValueError('Interest rate cannot be negative.')
        cls._monthly_interest_rate = value
        return f'New monthly interest rate = {round(cls._monthly_interest_rate * 100, 2)}%'
    
    @classmethod
    def _generate_confirmation(cls, account_no, tran_type):
        cls._tran_id += 1
        return (
            f"{tran_type}-{account_no}-"
            f"{datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S')}-"
            f"{cls._tran_id}"
        )
    
    @property
    def accnt_no(self):
        return self._accnt_no
    
    @accnt_no.setter
    def accnt_no(self, value):
        try:
            value = int(value)
        except:
            raise ValueError('Account number must be an integer value.')
        if value <= 0:
            raise ValueError('Account number must be greater than zero.')
        self._accnt_no = value
        
    @property
    def f_name(self):
        return self._f_name
    
    @f_name.setter
    def f_name(self, value):
        self._f_name = BankAccount._validate_name(value, 'First Name')
        
    @property
    def l_name(self):
        return self._l_name
    
    @l_name.setter
    def l_name(self, value):
        self._l_name = BankAccount._validate_name(value, 'Last Name')
        
    @property
    def full_name(self):
        return f'{self.f_name} {self.l_name}'
    
    @property
    def tz_offset(self):
        return self._tz_offset
    
    @tz_offset.setter
    def tz_offset(self, value):
        if value is None:
            value = TimeZone(datetime.timedelta(), 'UTC')
        if not isinstance(value, TimeZone):
            raise TypeError('tz_offset must be a TimeZone object.')
        self._tz_offset = value
        
    @property
    def balance(self):
        return self._balance
    
    def deposit(self, amount, interest=False):
        try:
            amount = round(Decimal(amount), 2)
        except:
            conf_no = self._generate_confirmation(self.accnt_no,
                                                  self._transaction_codes['rejected'])
            print('Deposit amount must be a decimal value.')
            print('Confirmation number:', conf_no)
            return
        if amount <= 0:
            conf_no = self._generate_confirmation(self.accnt_no,
                                                  self._transaction_codes['rejected'])
            print('Deposit amount must be greater than zero.')
            print('Confirmation number:', conf_no)
            return
        self._balance += amount
        if interest:
            conf_no = self._generate_confirmation(self.accnt_no,
                                              self._transaction_codes['interest'])
        else:
            conf_no = self._generate_confirmation(self.accnt_no,
                                              self._transaction_codes['deposit'])
        print(
            "Deposit accepted.\n"
            f"New balance = {self.balance}\n"
            f"Confirmation number = {conf_no}"
        )
        
    def withdrawal(self, amount):
        try:
            amount = round(Decimal(amount), 2)
        except:
            conf_no = self._generate_confirmation(self.accnt_no,
                                                  self._transaction_codes['rejected'])
            print('Withdrawal amount must be a decimal value.')
            print('Confirmation number:', conf_no)
            return
        if amount <= 0:
            conf_no = self._generate_confirmation(self.accnt_no,
                                                  self._transaction_codes['rejected'])
            print('Withdrawal amount must be greater than zero.')
            print('Confirmation number:', conf_no)
            return
        if self._balance - amount < 0:
            conf_no = self._generate_confirmation(self.accnt_no,
                                                  self._transaction_codes['rejected'])
            print('Insufficient funds.')
            print('Confirmation number:', conf_no)
            return
            
        self._balance -= amount
        conf_no = self._generate_confirmation(self.accnt_no,
                                              self._transaction_codes['withdrawal'])
        print(
            "Withdrawal accepted.\n"
            f"New balance = {self.balance}\n"
            f"Confirmation number = {conf_no}"
        )
    
    def interest_deposit(self):
        deposit_amount = self.balance * self.get_interest_rate()
        return self.deposit(deposit_amount, interest=True)
    
    def __repr__(self):
        return (
            f"""BankAccount
    Account No: {self.accnt_no}
    Full Name: {self.full_name}
    Timezone: {self.tz_offset}
    Balance: {self.balance}"""
        )

- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)

- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

In [320]:
delta = datetime.timedelta(hours=-3, minutes=30)

In [321]:
tz = TimeZone(delta, ' MST ')

In [322]:
tz.name

'MST'

In [323]:
ba = BankAccount(12345, 'Henry', 'Kissinger', tz, 5000)

Deposit accepted.
New balance = 5000.00
Confirmation number = D-12345-20210306234944-1


In [324]:
ba.set_interest_rate(0.0035)

'New monthly interest rate = 0.35%'

In [325]:
ba.set_interest_rate(-0.005)

ValueError: Interest rate cannot be negative.

In [326]:
ba.tz_offset

TimeZone(offset=-1 day, 21:30:00, name=MST)

In [327]:
ba._generate_confirmation(12345, 'D')

'D-12345-20210306234944-2'

In [328]:
bb = BankAccount(54321, 'Oswald', 'Cobblepot')

In [329]:
bb.tz_offset

TimeZone(offset=0:00:00, name=UTC)

In [330]:
bb._generate_confirmation(54321, 'W')

'W-54321-20210306234945-3'

In [331]:
ba.deposit(23)

Deposit accepted.
New balance = 5023.00
Confirmation number = D-12345-20210306234945-4


In [332]:
ba.deposit(300)

Deposit accepted.
New balance = 5323.00
Confirmation number = D-12345-20210306234945-5


In [333]:
ba.withdrawal('hello')

Withdrawal amount must be a decimal value.
Confirmation number: X-12345-20210306234945-6


In [334]:
ba.deposit(0.1)

Deposit accepted.
New balance = 5323.10
Confirmation number = D-12345-20210306234945-7


In [335]:
bb.deposit(-6)

Deposit amount must be greater than zero.
Confirmation number: X-54321-20210306234945-8


In [336]:
ba.interest_deposit()

Deposit accepted.
New balance = 5341.73
Confirmation number = I-12345-20210306234945-9


In [337]:
BankAccount.set_interest_rate(0.004)

'New monthly interest rate = 0.40%'

In [338]:
ba.interest_deposit()

Deposit accepted.
New balance = 5363.10
Confirmation number = I-12345-20210306234947-10


In [339]:
bb.interest_deposit()

Deposit amount must be greater than zero.
Confirmation number: X-54321-20210306234947-11


In [248]:
ba

BankAccount
    Account No: 12345
    Full Name: Henry Kissinger
    Timezone: TimeZone(offset=-1 day, 21:30:00, name=MST)
    Balance: 5344.39

In [185]:
pcn = BankAccount.parse_conf_no('X-54321-20210306025426-10', tz)

In [186]:
pcn

ConfirmationNumber(account_number='54321', transaction_code='X', transaction_id='10', time='2021-03-06 06:24:26 (MST)', time_utc='2021-03-06T02:54:26')

In [187]:
ba.interest_deposit()

Deposit accepted.
New balance = 5365.77
Confirmation number = D-12345-20210306030404-11


In [188]:
ba.balance

Decimal('5365.77')

In [154]:
ba = BankAccount('12345', ['h', 'e', 'l', 'l', 'o'], 'z')

ValueError: First name must contain only letters, spaces, or apostrophes.

In [155]:
ba.accnt_no

12345

In [156]:
ba.accnt_no = 'hello'

ValueError: Account number must be an integer value.

In [157]:
ba.accnt_no

12345

In [158]:
ba.__dict__

{'_accnt_no': 12345,
 '_f_name': 'Henry',
 '_l_name': 'Kissinger',
 '_tz_offset': TimeZone(offset=3:30:00, name=MST),
 '_balance': Decimal('5365.77')}

In [159]:
ba.l_name = 'Van der Lynden'

In [160]:
ba.f_name = "O'Shaughnessey"

In [161]:
ba.l_name

'Van der Lynden'

In [162]:
ba.f_name

"O'Shaughnessey"

In [163]:
ba.full_name

"O'Shaughnessey Van der Lynden"

In [164]:
all(map(lambda x: x.lower() in ascii_lowercase + " '", test_string))

NameError: name 'test_string' is not defined

In [165]:
import datetime

In [166]:
td = datetime.timedelta(hours=5, minutes=30)

In [62]:
type(td)

datetime.timedelta

In [32]:
td.seconds

19800

In [33]:
td.total_seconds()

19800.0

In [54]:
steve_time = datetime.timezone(td, 'Steve Time')

In [60]:
steve_time.

AttributeError: 'datetime.timezone' object has no attribute 'td'

In [41]:
test_time = datetime.datetime.utcnow()

In [42]:
test_time

datetime.datetime(2021, 3, 5, 20, 54, 20, 26361)

In [120]:
test_dec = Decimal(.234)

In [129]:
round(test_dec, 2)

Decimal('0.23')

In [265]:
test_time = datetime.datetime.utcnow()

In [276]:
test_time.strftime('%Y%m%d%H%M%S')

'20210305223547'

In [434]:
try:
    1 / 0
    print('hello')
except:
    raise ValueError('darn') from None
finally:
    print('test')

test


ValueError: darn

In [435]:
try:
    1 / 0
    print('hello')
except:
    raise ValueError('darn')
finally:
    print('test')

test


ValueError: darn

In [1]:
from collections import namedtuple

In [4]:
ConfirmationNumber = namedtuple('ConfirmationNumber',
                               ['account_number',
                               'transaction_code',
                               'transaction_id',
                               'time',
                               'time_utc'])

In [5]:
conf_no = ConfirmationNumber(12345, 'D', 5, 20210305124536, 20210305184536)

In [6]:
conf_no

ConfirmationNumber(account_number=12345, transaction_code='D', transaction_id=5, time=20210305124536, time_utc=20210305184536)

In [7]:
conf_no.time

20210305124536

In [10]:
tz = TimeZone(-7, 'MST')

TypeError: Offset must be a datetime.timedelta object.

In [11]:
td = datetime.timedelta(hours=5, minutes=15)

In [12]:
tz = TimeZone(td, 'Steve Time')

In [13]:
tz.name

'Steve Time'

In [14]:
tz.offset

datetime.timedelta(seconds=18900)

In [15]:
tz

TimeZone(offset=5:15:00, name=Steve Time)

In [16]:
td

datetime.timedelta(seconds=18900)

In [17]:
str(td)

'5:15:00'

In [21]:
tz.offset.min

datetime.timedelta(days=-999999999)

In [22]:
td.seconds

18900

In [27]:
datetime.datetime.utcnow() - td

datetime.datetime(2021, 3, 5, 21, 14, 48, 894837)

In [28]:
td = datetime.timedelta(hours=-5)

In [35]:
datetime.datetime.utcnow()

datetime.datetime(2021, 3, 6, 2, 31, 48, 207794)

In [37]:
test_datetime = datetime.datetime.strptime('20210306000509', '%Y%m%d%H%M%S')

In [38]:
test_datetime

datetime.datetime(2021, 3, 6, 0, 5, 9)

In [45]:
(test_datetime + td).isoformat()

'2021-03-05T19:05:09'

In [47]:
test_conf_no = 'D-12345-20210306000509-9'

In [48]:
test_conf_no.split('-')

['D', '12345', '20210306000509', '9']

In [274]:
datetime.timedelta()

datetime.timedelta(0)